In [14]:
from pyspark import SparkConf,SparkContext
from pyspark import rdd

In [15]:
conf=SparkConf().setMaster("local[*]").setAppName("First_APP")

In [17]:
sc=SparkContext(conf=conf)

In [4]:
data=sc.parallelize(range(10))
ans=data.reduce(lambda x,y:x+y)
print (ans)

45


In [5]:
lines=sc.textFile("E:\Spark\spark-2.1.0-bin-hadoop2.7\README.md")
print (lines.count())
print (lines.first())

104
# Apache Spark


In [18]:
class MyClass(object):
    def __init__(self):
        self.field = 1

    def doStuff(self, rdd):
        #将需要的字段提取到局部变量中即可
        field = self.field
        #return rdd.map(lambda s: self.field + x)是错误的
        #如果你传递的对象是某个对象的成员，或者在某个函数中引用了一个整个字段，会报错
        return rdd.map(lambda s: field + s)

In [26]:
my=MyClass()
data=sc.parallelize(range(10))
my.doStuff(data)
print (my)


In [22]:
pairs=lines.map(lambda x:(x.split(" ")[0],x))

In [30]:
print (lines.collect()[1:10])

['', 'Spark is a fast and general cluster computing system for Big Data. It provides', 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that', 'supports general computation graphs for data analysis. It also supports a', 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,', 'MLlib for machine learning, GraphX for graph processing,', 'and Spark Streaming for stream processing.', '', '<http://spark.apache.org/>']


In [31]:
print (pairs.collect()[1:10])

[('', ''), ('Spark', 'Spark is a fast and general cluster computing system for Big Data. It provides'), ('high-level', 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that'), ('supports', 'supports general computation graphs for data analysis. It also supports a'), ('rich', 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,'), ('MLlib', 'MLlib for machine learning, GraphX for graph processing,'), ('and', 'and Spark Streaming for stream processing.'), ('', ''), ('<http://spark.apache.org/>', '<http://spark.apache.org/>')]


In [32]:
#用Python对第二个元素进行筛选
result = pairs.filter(lambda keyValue:len(keyValue[1]) < 20)
print (result.collect()[1:10])

[('', ''), ('', ''), ('', ''), ('', ''), ('', ''), ('', ''), ('##', '## Building Spark'), ('', ''), ('', '')]


In [95]:
#在Python中使用reduceByKey()和mapValues()计算每个键对应的平均值
data=sc.parallelize([1,2,3,4,5,3,2])
data=data.map(lambda x:(x,1))
data1=data.mapValues(lambda x:(x,1)) #mapValues只接收KV键对的RDD数据，map则不用
data2=data1.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])) #key相同的进行聚合


In [93]:
data.collect()

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (3, 1), (2, 1)]

In [90]:
data1.collect()[0:10]

[(1, (1, 1)),
 (2, (1, 1)),
 (3, (1, 1)),
 (4, (1, 1)),
 (5, (1, 1)),
 (3, (1, 1)),
 (2, (1, 1))]

In [96]:
data2.collect()[0:10]

[(4, (1, 1)), (1, (1, 1)), (5, (1, 1)), (2, (2, 2)), (3, (2, 2))]

In [99]:
#用Python实现单词计数
rdd=sc.textFile("E:\Spark\spark-2.1.0-bin-hadoop2.7\README.md")
words = rdd.flatMap(lambda x:x.split(" "))
result = words.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [107]:
words.collect()[0:10] #''代表哪一行是空的

['#', 'Apache', 'Spark', '', 'Spark', 'is', 'a', 'fast', 'and', 'general']

In [103]:
result.collect()[0:10]

[('', 72),
 ('guide,', 1),
 ('APIs', 1),
 ('optimized', 1),
 ('name', 1),
 ('contributing', 1),
 ('developing', 1),
 ('It', 2),
 ('package.', 1),
 ('particular', 2)]

In [132]:
#在Python中使用combineByKey()求每个键对应的平均值
nums=sc.parallelize([1,2,3,4,5,3,2])
nums=nums.map(lambda x:(x,1))
sumCount = nums.combineByKey((lambda x:(x,1)),
                             (lambda x,y:(x[0]+y,x[1]+1)),
                             (lambda x,y:(x[0]+y[0],x[1]+y[1])))
#sumCount.map(lambda key,xy:(key.xy[0]/xy[1])).collectAsMap()
#sumCount.map(lambda key,xy:(key.xy[0]/xy[1])).collectAsMap()

In [118]:
sumCount.collect()

[(4, (1, 1)), (1, (1, 1)), (5, (1, 1)), (2, (2, 2)), (3, (2, 2))]

In [127]:
data4=data1.reduceByKey(lambda x,y:(x[0]+1,x[1]+1))

In [128]:
data4.collect()

[(4, (1, 1)), (1, (1, 1)), (5, (1, 1)), (2, (2, 2)), (3, (2, 2))]

In [133]:
nums1=sc.parallelize([1,2,3,4,5,3,2])
nums2=nums1.map(lambda x:(x,1))
nums3=nums2.reduceByKey(lambda x,y:x+y)

In [134]:
nums2.collect()

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (3, 1), (2, 1)]

In [142]:
nums5=nums3.union(nums3)
nums3.collect()

[(4, 1), (1, 1), (5, 1), (2, 2), (3, 2)]

In [143]:
nums5.collect()

[(4, 1),
 (4, 1),
 (1, 1),
 (5, 1),
 (1, 1),
 (5, 1),
 (2, 2),
 (2, 2),
 (3, 2),
 (3, 2)]

combineByKey函数主要接受了三个函数作为参数，分别为createCombiner、mergeValue、mergeCombiners总体效果为RDD[(K,V)]=>RDD[(K,C)],使用用户设置好的聚合 函数对每个key中的value进行组合函数原型为： def combinByKey[C](createCombiner:V=>C,mergeValue:(C,V)=>C,mergeCombiners:(C,C)=>C:RDD[(K,C)])

In [144]:
#combineByKey函数主要接受了三个函数作为参数，分别为createCombiner、mergeValue、mergeCombiners
nums4 = nums5.combineByKey((lambda x:(x,1)),
                             (lambda x,y:(x[0]+y,x[1]+1)),
                             (lambda x,y:(x[0]+y[0],x[1]+y[1])))
nums4.mapValues(lambda x:float(x[0]/x[1])).collect()

[(4, 1.0), (1, 1.0), (5, 1.0), (2, 2.0), (3, 2.0)]

In [159]:
nums4.mapValues(lambda x:float(x[0]/x[1])).collectAsMap()

{1: 1.0, 2: 2.0, 3: 2.0, 4: 1.0, 5: 1.0}

In [145]:
nums4.collect()

[(4, (2, 2)), (1, (2, 2)), (5, (2, 2)), (2, (4, 2)), (3, (4, 2))]

In [148]:
#下面来细分combineByKey函数的效果
nums7=nums5.map(lambda x:(x,1))
nums7.collect()

[((4, 1), 1),
 ((4, 1), 1),
 ((1, 1), 1),
 ((5, 1), 1),
 ((1, 1), 1),
 ((5, 1), 1),
 ((2, 2), 1),
 ((2, 2), 1),
 ((3, 2), 1),
 ((3, 2), 1)]

In [161]:
#在Python中自定义reduceByKey()的并行度
data = [("a",3),("b",4),("a",1)]
sc.parallelize(data).reduceByKey(lambda x,y:x+y).collect()#默认并行度

[('b', 4), ('a', 4)]

In [162]:
sc.parallelize(data).reduceByKey(lambda x,y:x+y,10).collect()#自定义并行度

[('b', 4), ('a', 4)]

In [195]:
#sortByKey
#在Python中以字符串顺序对整数进行自定义排序

In [ ]:
#rdd=sc.parallelize([1,2,3,4])
rdd=sc.parallelize(data)
#rdd=rdd.map(lambda x:(x,1))
rdd1=rdd.sortByKey(ascending = True,numPartitions = None,keyfunc = lambda x: str(x))

In [196]:
rdd1.collect()

[('a', 3), ('a', 1), ('b', 4)]

下面来介绍转换与行动：

转换：转换的返回值是一个新的RDD集合，而不是单个值。调用一个变换方法，不会有任何求值计算，它只获取一个RDD作为参数，然后返回一个新的RDD。
行动：行动操作计算并返回一个新的值。当在一个RDD对象上调用行动函数时，会在这一时刻计算全部的数据处理查询并返回结果值。



----------------------------
Transformations:
----------------------------

combineByKey函数主要接受了三个函数作为参数，分别为createCombiner、mergeValue、mergeCombiners总体效果为RDD[(K,V)]=>RDD[(K,C)],使用用户设置好的聚合 函数对每个key中的value进行组合函数原型为： def combinByKey[C](createCombiner:V=>C,mergeValue:(C,V)=>C,mergeCombiners:(C,C)=>C:RDD[(K,C)])

In [232]:
nums1=sc.parallelize([1,2,3,4,5,3,2])
nums2=nums1.map(lambda x:(x,1))
nums3=nums2.reduceByKey(lambda x,y:x+y)
nums5=nums3.union(nums3)
nums3.collect()

[(4, 1), (1, 1), (5, 1), (2, 2), (3, 2)]

In [233]:
nums5.collect()

[(4, 1),
 (4, 1),
 (1, 1),
 (5, 1),
 (1, 1),
 (5, 1),
 (2, 2),
 (2, 2),
 (3, 2),
 (3, 2)]

In [234]:
#combineByKey函数主要接受了三个函数作为参数，分别为createCombiner、mergeValue、mergeCombiners
nums4 = nums5.combineByKey((lambda x:(x,1)),
                             (lambda x,y:(x[0]+y,x[1]+1)),
                             (lambda x,y:(x[0]+y[0],x[1]+y[1])))
nums4.mapValues(lambda x:float(x[0]/x[1])).collect()

[(4, 1.0), (1, 1.0), (5, 1.0), (2, 2.0), (3, 2.0)]

In [235]:
nums4.collect()

[(4, (2, 2)), (1, (2, 2)), (5, (2, 2)), (2, (4, 2)), (3, (4, 2))]

filter(func)  --- 返回一个新的数据集，这个数据集中的元素是通过func函数筛选后返回为true的元素（简单的说就是，对数据集中的每个元素进行筛选，如果符合条件则返回true，
不符合返回false，最后将返回为true的元素组成新的数据集返回）。


In [216]:
data1=[1,2,3,4,5]
rdd = sc.parallelize(data1).filter(lambda x:x%2==0)
rdd.collect()

[2, 4]

flatMap(func [, preservesPartitioning=False])  --- 类似于map(func)， 但是不同的是map对每个元素处理完后返回与原数据集相同元素数量的数据集，
而flatMap返回的元素数不一定和原数据集相同。


In [188]:
#### for flatMap()
rdd = sc.parallelize([2,3,4])
sorted(rdd.flatMap(lambda x: range(1,x)).collect())

[1, 1, 1, 2, 2, 3]

In [18]:
rdd = sc.parallelize([[2,3,4],[1,2,3]])
rdd.flatMap(lambda x: x).collect() #是按最小元素来映射的，在这里是每个元素的映射，而不是两个列表的映射

[2, 3, 4, 1, 2, 3]

In [189]:
sorted(rdd.flatMap(lambda x:[(x,x), (x,x)]).collect())

[(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]

In [204]:
#### for map()
rdd = sc.parallelize([2,3,4])
rdd.map(lambda x: range(1,x)).collect()

[range(1, 2), range(1, 3), range(1, 4)]

In [202]:
sorted(rdd.map(lambda x:[(x,x), (x,x)]).collect())

[[(2, 2), (2, 2)], [(3, 3), (3, 3)], [(4, 4), (4, 4)]]

mapPartitions(func [, preservesPartitioning=False])  
mapPartitions是map的一个变种。map的输入函数是应用于RDD中每个元素，而mapPartitions的输入函数是应用于每个分区，也就是把每个分区中的内容作为整体来处理的

In [206]:
rdd = sc.parallelize([1,2,3,4,5], 3)
def f(iterator): yield sum(iterator) #分区求和
rdd.mapPartitions(f).collect()

[1, 5, 9]

In [207]:
#mapPartitionsWithIndex生成分区的索引
rdd = sc.parallelize([1,2,3,4,5], 3)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).collect()

[0, 1, 2]

reduceByKey(func [, numPartitions=None, partitionFunc=<function portable_hash at 0x7fa664f3cb90>]) 
reduceByKey就是对元素为kv对的RDD中Key相同的元素的value进行reduce，因此，key相同的多个元素的值被reduce为一个值，然后与原RDD中的key组成一个新的kv对。

In [209]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())
#或者 sorted(rdd.reduceByKey(lambda a,b:a+b).collect())

[('a', 2), ('b', 1)]

sortByKey([ascending=True, numPartitions=None, keyfunc=<function <lambda> at 0x7fa665048c80>])
返回排序后的数据集。该函数就是队kv对的RDD数据进行排序，keyfunc是对key进行处理的函数，如非需要，不用管。


In [215]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('D', 4)]
sc.parallelize(tmp).sortByKey(True, 1).collect()

[('1', 3), ('D', 4), ('a', 1), ('b', 2)]

In [212]:
sc.parallelize(tmp).sortByKey(True, 2, keyfunc=lambda k:k.lower()).collect()
#注意，比较两个结果可看出，keyfunc对键的处理只是在数据处理的过程中起作用，不能真正的去改变键名

[('1', 3), ('a', 1), ('b', 2), ('D', 4)]

join(otherDataset [, numPartitions=None]) 
join就是对元素为kv对的RDD中key相同的value收集到一起组成(v1,v2)，然后与原RDD中的key组合成一个新的kv对，返回。

In [213]:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
sorted(x.join(y).collect())

[('a', (1, 2)), ('a', (1, 3))]

cartesian(otherDataset)  --- 返回一个笛卡尔积的数据集，这个数据集是通过计算两个RDDs得到的。

In [214]:
x = sc.parallelize([1,2,3])
y = sc.parallelize([4,5])
x.cartesian(y).collect()

[(1, 4), (1, 5), (2, 4), (2, 5), (3, 4), (3, 5)]

sample(withReplacement, fraction, seed) 
从数据中抽样，withReplacement表示是否有放回，withReplacement=true表示有放回抽样，fraction为抽样的概率（0<=fraction<=1），seed为随机种子。 
例如：从1-100之间抽取样本，被抽取为样本的概率为0.2

！！！注意，Spark中的sample抽样，当withReplacement=True时，相当于采用的是泊松抽样；当withReplacement=False时，相当于采用伯努利抽样，fraction并不是表示抽样得到的样本占原来数据总量的百分比，而是一个元素被抽取为样本的概率。fraction=0.2并不是说明要抽出100个数字中20%的数据作为样本，而是每个数字被抽取为样本的概率为0.2，这些数字被认为来自同一总体，样本的大小并不是固定的，而是服从二项分布。

In [249]:
data = sc.parallelize(range(1,101),2)
sampleData = data.sample(True, 0.2)
sampleData.count()

23

In [250]:
sampleData.collect()

[7,
 24,
 25,
 29,
 33,
 39,
 42,
 45,
 46,
 54,
 60,
 64,
 69,
 70,
 79,
 82,
 83,
 89,
 91,
 93,
 95,
 95,
 97]

intersection(otherDataset) 
交集操作，将源数据集与union中的输入数据集取交集，并返回新的数据集。

In [251]:
data1 = sc.parallelize(range(10))
data2 = sc.parallelize(range(6,15))
data1.intersection(data2).collect()

[8, 9, 6, 7]

distinct([numTasks]) 去除数据集中的重复元素。

In [252]:
data1 = sc.parallelize(range(10))
data2 = sc.parallelize(range(6,15))
data1.union(data2).distinct().collect()

[0, 8, 1, 9, 2, 10, 11, 3, 12, 4, 5, 13, 14, 6, 7]

--------------------------------------------
Action:
--------------------------------------------

reduce(func)  --- reduce将RDD中元素两两传递给输入函数，同时产生一个新的值，新产生的值与RDD中下一个元素再被传递给输入函数直到最后只有一个值为止。

In [217]:
from operator import add
sc.parallelize([1,2,3,4,5]).reduce(add)

15

collect()  --- 返回RDD中的数据，以list形式。

In [218]:
sc.parallelize([1,2,3,4,5]).collect()

[1, 2, 3, 4, 5]

count()  --- 返回RDD中的元素个数。

In [220]:
sc.parallelize([1,2,3,4,5]).count()

5

first()  --- 返回RDD中的第一个元素。

In [221]:
sc.parallelize([1,2,3,4,5]).first()

1

take(n)  --- 返回RDD中前n个元素。

In [222]:
sc.parallelize([1,2,3,4,5]).take(2)

[1, 2]

takeOrdered(n [, key=None])  --- 返回RDD中前n个元素，但是是升序(默认)排列后的前n个元素，或者是通过key函数指定后的RDD

In [223]:
sc.([9,7,3,2,6,4]).takeOrdered(3)

[2, 3, 4]

In [224]:
sc.parallelize([9,7,3,2,6,4]).takeOrdered(3, key=lambda x:-x)

[9, 7, 6]

saveAsTextFile(path [, compressionCodecClass=None])  --- 该函数将RDD保存到文件系统里面，并且将其转换为文本行的文件中的每个元素调用 tostring 方法。
parameters:  path - 保存于文件系统的路径

In [225]:
tempFile = NamedTemporaryFile(delete=True)
tempFile.close()
sc.parallelize(range(10)).saveAsTextFile(tempFile.name)
from fileinput import input
from glob import glob
''.join(sorted(input(glob(tempFile.name + "/part-0000*"))))

NameError: name 'NamedTemporaryFile' is not defined

countByKey()  --- 返回一个字典（key,count），该函数操作数据集为kv形式的数据，用于统计RDD中拥有相同key的元素个数。

In [226]:
defdict = sc.parallelize([("a",1), ("b",1), ("a", 1)]).countByKey()
defdict

defaultdict(int, {'a': 2, 'b': 1})

In [227]:
defdict.items()

dict_items([('b', 1), ('a', 2)])

countByValue()  --- 返回一个字典（value,count），该函数操作一个list数据集，用于统计RDD中拥有相同value的元素个数。

In [228]:
sc.parallelize([1,2,3,1,2,5,3,2,3,2]).countByValue().items()

dict_items([(1, 2), (2, 4), (3, 3), (5, 1)])

foreach(func)  --- 运行函数func来处理RDD中的每个元素，这个函数常被用来updating an Accumulator或者与外部存储系统的交互。

In [230]:
def f(x): print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreach(f)
#note: 打印是随机的，并不是一定按1,2,3,4,5的顺序打印

collectAsMap:以rdd元素为元组，以元组中一个元素作为索引返回RDD中的数据

In [239]:
m = sc.parallelize([('a', 2), (3, 4)]).collectAsMap()
print (m)

{3: 4, 'a': 2}


groupby函数：根据提供的方法为RDD分组：

In [5]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
def fun(i):
    return i % 2

result=rdd.groupBy(fun).collect()
print (result)

[(0, <pyspark.resultiterable.ResultIterable object at 0x000002418A4985C0>), (1, <pyspark.resultiterable.ResultIterable object at 0x000002418A4985F8>)]


In [6]:
print ([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]


数据分区：数据比较大时，可以用partitionBy()转化为哈希分区。即通过向partitionBy传递一个spark.HashPartitioner对象来实现该操作。在Python中不能将HashPartitioner对象传递给partitionBy，只需要把需要的分区数传递过去（如 rdd.partitionBy(100)）。

　　在spark中，会为生成的结果RDD设好分区方式的操作有：cogroup(),groupWith(),join(),leftOuterJoin(),rightOutJoin,groupByKey(),reduceByKey(),combineByKey(),partitionBy(),sort(),mapValues(),flatMapValues(),filter()。最后三种只有当父RDD有分区方式时，结果RDD才会有分区RDD。其他的操作生成的结果都不会存在特定的分区方式。


In [13]:
a=[1,2,3,2]
a.sort()

In [1]:
a1=[(1,2),(2,3),(0,9),(3,4)]
print (type (a1))
sorted(a1,key=lambda x:x[1])

<class 'list'>


[(1, 2), (2, 3), (3, 4), (0, 9)]

In [1]:
import numpy as np
a=np.zeros(2)
print (a)

[ 0.  0.]
